In [26]:
import elabapi_python

# Initialize a configuration object from the library
configuration = elabapi_python.Configuration()
# Set the host
configuration.host = "https://elabftw.michaelscheidegger.ch/api/v2"
# Create an API client object with our configuration
api_client = elabapi_python.ApiClient(configuration)
# Set the Api Key in Authorization header
api_client.set_default_header(header_name='Authorization', header_value="1-eebe876e1a5b87f95ed7e1b6af3132a158a4225a8d1a22214693ccb077de16ad3342b4b5c2cca4e4b5021")

# Depending on your needs, instantiate the correct object with api_client as argument
info_client = elabapi_python.InfoApi(api_client)
exp_client = elabapi_python.ExperimentsApi(api_client)
items_client = elabapi_python.ItemsApi(api_client)

print("ElabFTW API Client initialized.")
print("API Client Host: ", configuration.host)
print("info_client: ", info_client)
print("exp_client: ", exp_client.read_experiments())
print("items_client: ", items_client.read_items())

ElabFTW API Client initialized.
API Client Host:  https://elabftw.michaelscheidegger.ch/api/v2
info_client:  <elabapi_python.api.info_api.InfoApi object at 0x000001DFE8BD8690>
exp_client:  [{'_date': '2025-04-18',
 'access_key': None,
 'body': '<h1>My modified experiment</h1><p>This experiment was created using '
         'the ElabFTW API, passing the Experiment object directly.</p><p>Some '
         '<strong>bold text</strong> and <em>italic text</em>.</p>',
 'body_html': None,
 'canread': '{"base": 30, "teams": [], "users": [], "teamgroups": []}',
 'canwrite': '{"base": 20, "teams": [], "users": [], "teamgroups": []}',
 'category': 1,
 'category_color': 'a85100',
 'category_title': 'Kombucha',
 'comments': None,
 'content_type': 1,
 'created_at': '2025-04-18 17:43:38',
 'custom_id': None,
 'elabid': '20250418-7a0a991b96d94c7dbc20db054683b1564d30e802',
 'exclusive_edit_mode': None,
 'experiments_links': None,
 'firstname': 'Michael',
 'fullname': 'Michael Scheidegger',
 'has_attacheme

In [36]:
# --- Create a New Experiment ---
print("\nCreating a new experiment...")
# Create an instance of the Experiment model
experiment = elabapi_python.Experiment()

# Set the properties of the experiment
experiment.title = "My second experiment with API"
# Set the body content using HTML formatting
experiment.body = "<h1>My modified experiment</h1><p>This experiment was created using the ElabFTW API, passing the Experiment object directly.</p><p>Some <strong>bold text</strong> and <em>italic text</em>.</p>"

experiment.content_type = 1 # 1 = HTML, 2 = Markdown

experiment.category = 1 
experiment.status = 1
experiment.tags = ["API", "Python"]

# --- Post the Experiment ---
try:
    # Call the post_experiment method, passing the experiment *object* directly
    # The library handles serializing the object into the correct JSON format
    print(f"Posting experiment titled: {experiment.title}")
    response_data, status_code, headers = exp_client.post_experiment_with_http_info(body=experiment) # Pass the object here
    print("\nExperiment created successfully!")

    print(f"Response status code: {status_code}")
    print(f"Response headers: {headers}")
    print(f"Response data: {response_data}")

    location = headers.get('Location')
    exp_id = int(location.split('/').pop())

    print(f"Experiment ID: {exp_id}")

    experiment = exp_client.get_experiment(exp_id)

    print(f"[*] Our experiment now has this title: {experiment.title}")
    print(f"[*] Check it out on the web interface: {experiment.sharelink}")


except elabapi_python.ApiException as e:
    # Handle potential API errors (e.g., authentication issues, invalid data)
    print(f"\nError creating experiment: {e.status} {e.reason}")
    print(f"Response body: {e.body}")
except Exception as e:
    # Handle other potential errors (e.g., network issues)
    print(f"\nAn unexpected error occurred: {e}")

print("\nScript finished.")


Creating a new experiment...
Posting experiment titled: My second experiment with API

Experiment created successfully!
Response status code: 201
Response headers: HTTPHeaderDict({'Access-Control-Allow-Credentials': 'true', 'Access-Control-Expose-Headers': 'Location, Content-Encoding, Content-Disposition, Cache-Control', 'Cache-Control': 'max-age=0, private, must-revalidate, no-cache, private', 'Content-Security-Policy': "default-src 'self' data:; script-src 'self' ; connect-src 'self' blob: https://get.elabftw.net; img-src 'self' data: blob:; style-src 'self' 'unsafe-inline'; font-src 'self' data:; object-src 'self'; base-uri 'none'; frame-ancestors 'none'", 'Content-Type': 'text/html; charset=UTF-8', 'Date': 'Fri, 18 Apr 2025 16:43:05 GMT', 'Location': 'https://elabftw.michaelscheidegger.ch/api/v2/experiments/32', 'Permissions-Policy': "autoplay 'none'; camera 'self'; document-domain 'none'; encrypted-media 'none'; fullscreen 'self'; geolocation 'none'; microphone 'self'; midi 'none

In [37]:
# Block 3: Modify the Experiment (Corrected)

# --- Define the changes ---
# You only want to change the body in this example
new_body_content = "<h1>My MODIFIED experiment Body</h1><p>This experiment was MODIFIED using the ElabFTW API's PATCH method.</p><p>Some <strong>bold text</strong> and <em>italic text</em>.</p>"
# If you also wanted to change the title, uncomment the line below
# new_title = "My modified experiment title via PATCH"

# --- Create the update payload dictionary ---
# This dictionary contains ONLY the fields to be updated
update_payload = {
    'body': new_body_content
    # If changing the title, add it here:
    # , 'title': new_title
}

# Ensure exp_id is defined from Block 2 (the creation block)
if 'exp_id' in locals() or 'exp_id' in globals():
    print(f"\nAttempting to modify experiment with ID: {exp_id}")

    try:
        # --- Patch the Experiment ---
        # Pass the update_payload dictionary as the body for the PATCH request
        response_data, status_code, headers = exp_client.patch_experiment_with_http_info(
            id=exp_id,
            body=update_payload
        )

        print("\nExperiment modified successfully!")
        print(f"[*] Status Code: {status_code}")

        # IMPORTANT: Use 'response_data' for the updated information.
        # 'response_data' is the Experiment object returned by the server AFTER the update.
        # The old 'experiment' variable still holds the state BEFORE the patch.
        print(f"[*] The experiment now has this title: {response_data.title}")
        print(f"[*] Check it out on the web interface: {response_data.sharelink}")
        # print(f"[*] Updated Body Snippet: {response_data.body[:100]}...") # Optional: check body update

    except elabapi_python.ApiException as e:
        # Handle potential API errors
        print(f"\nError modifying experiment: {e.status} {e.reason}")
        # Print more details if available in the exception body
        if e.body:
            import json
            try:
                error_details = json.loads(e.body)
                print(f"Error details: {json.dumps(error_details, indent=2)}")
            except json.JSONDecodeError:
                print(f"Raw error body: {e.body}")
        else:
            print(f"Raw exception: {e}")
    except NameError:
        print("\nError: 'exp_id' is not defined. Make sure Block 2 ran successfully.")
    except Exception as e:
        # Handle other potential errors
        print(f"\nAn unexpected error occurred during modification: {e}")

else:
    print("\nCannot run Block 3: 'exp_id' was not found. Please run Block 2 first.")


Attempting to modify experiment with ID: 32

Experiment modified successfully!
[*] Status Code: 200
[*] The experiment now has this title: My second experiment with API
[*] Check it out on the web interface: https://elabftw.michaelscheidegger.ch/experiments.php?mode=view&id=32


In [38]:
# Block 2: Create a New Experiment (with Body Verification)

# --- Create a New Experiment ---
print("\nCreating a new experiment...")
# Create an instance of the Experiment model
new_experiment_data = elabapi_python.Experiment()

# Set the properties of the experiment
new_experiment_data.title = "My experiment created with initial body"
# Set the body content using HTML formatting
initial_body_content = "<h1>Initial Body Content</h1><p>This body was set during the creation (POST) process.</p><p>Some <strong>bold text</strong> and <em>italic text</em>.</p>"
new_experiment_data.body = initial_body_content

new_experiment_data.content_type = 1 # 1 = HTML, 2 = Markdown

# Assuming category ID 1 and status ID 1 exist in your instance
new_experiment_data.category = 1
new_experiment_data.status = 1
new_experiment_data.tags = ["API", "Python", "CreationTest"]

# --- Post the Experiment ---
exp_id = None # Initialize exp_id
try:
    # Call the post_experiment method, passing the experiment *object* directly
    # The library handles serializing the object into the correct JSON format
    print(f"Posting experiment titled: {new_experiment_data.title}")

    # Pass the new_experiment_data object containing title, body, etc.
    response_data_post, status_code_post, headers_post = exp_client.post_experiment_with_http_info(
        body=new_experiment_data
    )
    print("\nExperiment creation API call successful!")
    print(f"Response status code: {status_code_post}")

    # --- Verification Step 1: Extract ID ---
    location = headers_post.get('Location')
    if location:
        exp_id = int(location.split('/').pop())
        print(f"Successfully extracted Experiment ID: {exp_id}")

        # --- Verification Step 2: Fetch the created experiment from server ---
        print(f"\nFetching experiment {exp_id} back from server for verification...")
        # Use a different variable name to avoid confusion with the initial data object
        fetched_experiment = exp_client.get_experiment(exp_id)
        print("Successfully fetched experiment data.")

        # --- Verification Step 3: Check the fetched data ---
        print(f"[*] Fetched Title: {fetched_experiment.title}")
        print(f"[*] Fetched Sharelink: {fetched_experiment.sharelink}")

        # Check if the body content was saved correctly
        if fetched_experiment.body:
            print(f"[*] Fetched Body is present.")
            # Optional: Print a snippet to compare
            print(f"[*] Fetched Body Snippet: {fetched_experiment.body[:150]}...")
            if fetched_experiment.body == initial_body_content:
                print("[*] Fetched Body matches initial content. SUCCESS!")
            else:
                print("[!] WARNING: Fetched Body does NOT match initial content.")
        else:
            print("[!] WARNING: Fetched Body is EMPTY or None.")

    else:
        print("\nError: Could not retrieve Location header from POST response. Cannot verify experiment.")
        # Optional: print response details for debugging
        # print(f"Response headers: {headers_post}")
        # print(f"Response data: {response_data_post}")


except elabapi_python.ApiException as e:
    # Handle potential API errors during POST
    print(f"\nError creating experiment: {e.status} {e.reason}")
    print(f"Response body: {e.body}")
except Exception as e:
    # Handle other potential errors (network, ID extraction, GET request)
    print(f"\nAn unexpected error occurred during creation or verification: {e}")

if exp_id:
    print(f"\nBlock 2 finished. Experiment ID {exp_id} should be created with its initial body.")
else:
    print("\nBlock 2 finished, but experiment creation or verification failed.")


Creating a new experiment...
Posting experiment titled: My experiment created with initial body

Experiment creation API call successful!
Response status code: 201
Successfully extracted Experiment ID: 33

Fetching experiment 33 back from server for verification...
Successfully fetched experiment data.
[*] Fetched Title: My experiment created with initial body
[*] Fetched Sharelink: https://elabftw.michaelscheidegger.ch/experiments.php?mode=view&id=33
[!] WARNING: Fetched Body is EMPTY or None.

Block 2 finished. Experiment ID 33 should be created with its initial body.


In [49]:
import elabapi_python
from elabapi_python.rest import ApiException
import json 
import logging

# Set up logging configuration
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logging.getLogger().setLevel(logging.DEBUG)

def create_and_update_experiment(
    exp_client: elabapi_python.ExperimentsApi,
    title: str,
    body: str,
    category_id: int,
    status_id: int = 1,
    tags: list = None,
    content_type: int = 1  # 1 = HTML, 2 = Markdown
) -> elabapi_python.Experiment | None:
    """
    Creates a new experiment and immediately updates it with details.

    This function implements a workaround for cases where setting the body
    during initial creation (POST) might not work as expected. It first
    creates a minimal experiment (with title, category, status) and then
    immediately updates (PATCH) it to add the body, tags, and content_type.

    Args:
        exp_client: An initialized ExperimentsApi client instance.
        title: The desired title for the experiment.
        body: The HTML or Markdown content for the experiment body.
        category_id: The ID of the experiment category.
        status_id: The ID of the experiment status.
        tags: A list of strings for tags (optional).
        content_type: 1 for HTML (default), 2 for Markdown.

    Returns:
        The fully created and updated Experiment object if successful,
        otherwise None.
    """
    if tags is None:
        tags = []

    exp_id = None
    logging.debug(f"\n--- Starting Create-Then-Update Process for title: '{title}' ---")

    # === Step 1: Create Minimal Experiment (POST) ===
    try:
        logging.debug("[Step 1] Creating minimal experiment...")
        minimal_experiment = elabapi_python.Experiment(
            title=f"{title}",
            category=category_id,
            status=status_id,
            tags=tags,
            content_type=content_type
        )
        response_post, status_post, headers_post = exp_client.post_experiment_with_http_info(
            body=minimal_experiment, async_req=False
        )
        logging.debug(f"[Step 1] POST call successful (Status: {status_post}).")

        location = headers_post.get('Location')
        if location:
            exp_id = int(location.split('/').pop())
            logging.debug(f"[Step 1] Successfully extracted Experiment ID: {exp_id}")
        else:
            logging.debug("[Step 1] CRITICAL ERROR: 'Location' header missing in POST response. Cannot proceed.")
            # print(f"POST Response Headers: {headers_post}") # Uncomment for debug
            return None

    except ApiException as e:
        logging.debug(f"[Step 1] API Error during initial creation: {e.status} {e.reason}")
        if e.body:
            try: logging.error(f"Error details: {json.dumps(json.loads(e.body), indent=2)}")
            except: logging.error(f"Raw error body: {e.body}")
        return None
    except Exception as e:
        logging.debug(f"[Step 1] Unexpected error during initial creation: {e}")
        return None

    # === Step 2: Update Experiment with Details (PATCH) ===
    if exp_id:
        try:
            logging.debug(f"[Step 2] Updating experiment {exp_id} with full details...")
            update_payload = {
                'body': body,          # Set the body content
            }
            exp_client.patch_experiment_with_http_info(
                id=exp_id,
                body=update_payload,
                async_req=False
            )

        except ApiException as e:
            logging.debug(f"[Step 2] API Error during update: {e.status} {e.reason}")
            if e.body:
                try: print(f"Error details: {json.dumps(json.loads(e.body), indent=2)}")
                except: print(f"Raw error body: {e.body}")
            logging.debug("[Step 2] Update failed. The experiment might exist in a minimal state.")
            # Optionally try fetching the minimal experiment anyway
            try:
                 minimal_exp = exp_client.get_experiment(exp_id)
                 logging.error("[Step 2] Returning the minimally created experiment before update failed.")
                 return minimal_exp
            except:
                 return None # Fetching also failed
        except Exception as e:
            print(f"[Step 2] Unexpected error during update: {e}")
            return None # Or try to return minimal experiment as above

    # === Step 3: Fetch Final Experiment State (GET) ===
    if exp_id:
        try:
            logging.debug(f"[Step 3] Fetching final state of experiment {exp_id}...")
            final_experiment = exp_client.get_experiment(exp_id)
            logging.debug("[Step 3] Successfully fetched final experiment.")
            logging.debug(f"--- Create-Then-Update Process for ID {exp_id} COMPLETE ---")
            return final_experiment # Success! Return the final object

        except ApiException as e:
            logging.error(f"[Step 3] API Error fetching final state: {e.status} {e.reason}")
            if e.body:
                try: logging.error(f"Error details: {json.dumps(json.loads(e.body), indent=2)}")
                except: logging.error(f"Raw error body: {e.body}")
            print("[Step 3] Could not fetch final state, but experiment might have been updated.")
            return None
        except Exception as e:
            logging.error(f"[Step 3] Unexpected error fetching final state: {e}")
            return None

    # Should not be reached if logic is correct, but as a fallback
    return None

# --- Example Usage ---
# Assuming 'exp_client' is already initialized from Block 1

# Define experiment details
my_title = "Experiment via Create-Update Function"
my_body = "<h1>Function Workaround</h1><p>This body was set via the PATCH step of the function.</p>"
my_category = 1 # Replace with your actual category ID
my_status = 1   # Replace with your actual status ID
my_tags = ["Workaround", "Function", "API"]

# Call the function
created_experiment = create_and_update_experiment(
    exp_client=exp_client,
    title=my_title,
    body=my_body,
    category_id=my_category,
    status_id=my_status,
    tags=my_tags,
    content_type=1 # HTML
)

# Check the result
if created_experiment:
    print("\n--- Function Result ---")
    print(f"Successfully created/updated experiment.")
    print(f"ID: {created_experiment.id}")
    print(f"Title: {created_experiment.title}")
    print(f"Body Snippet: {created_experiment.body[:100]}...")
    print(f"Sharelink: {created_experiment.sharelink}")
else:
    print("\n--- Function Result ---")
    print("Function failed to create and update the experiment.")

DEBUG:root:
--- Starting Create-Then-Update Process for title: 'Experiment via Create-Update Function' ---
DEBUG:root:[Step 1] Creating minimal experiment...
DEBUG:root:[Step 1] POST call successful (Status: 201).
DEBUG:root:[Step 1] Successfully extracted Experiment ID: 44
DEBUG:root:[Step 2] Updating experiment 44 with full details...
DEBUG:root:[Step 3] Fetching final state of experiment 44...
DEBUG:root:[Step 3] Successfully fetched final experiment.
DEBUG:root:--- Create-Then-Update Process for ID 44 COMPLETE ---



--- Function Result ---
Successfully created/updated experiment.
ID: 44
Title: Experiment via Create-Update Function
Body Snippet: <h1>Function Workaround</h1><p>This body was set via the PATCH step of the function.</p>...
Sharelink: https://elabftw.michaelscheidegger.ch/experiments.php?mode=view&id=44
